# 1. Import libraries

In [1]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import tensorflow as tf
from konlpy.tag import Twitter
from collections import Counter
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt

keras = tf.keras
t = Twitter()

/home/ubuntu/anaconda3/envs/juns/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/home/ubuntu/anaconda3/envs/juns/lib/python3.6/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


# 2. fit tokenizer to our datasets

In [3]:
from vectorizer import BaseVectorizer
tokenizer = BaseVectorizer(t.morphs)

In [4]:
df = pd.read_csv('./Intent/train_intent.csv')

In [5]:
df[500:520]

,question,intent
500,여자 영어 말해줄래요,번역
501,고장 영어 뭔데요,번역
502,방문 영어 뭔지 알아요,번역
503,코끼리 영어 뭔지 알려줄래요,번역
504,파인애플 영어 번역요,번역
505,배 뭔지 영어 번역해줘요,번역
506,전기 번역해주세요,번역
507,물고기 영어 번역해줄래요,번역
508,이 문장 영어 말해줘요,번역
509,이 단어 영어 말해주세요,번역


In [6]:
tokenizer.fit(df['question'].values)

scanning was done                                        
1405 terms are recognized


In [7]:
tokenizer.vocabulary_

{'_PAD_': 0,
 '_UNK_': 1,
 '_STA_': 2,
 '_EOS_': 3,
 '오늘': 4,
 '알려줘': 5,
 '명언': 6,
 '이': 7,
 '누구': 8,
 '알려줘요': 9,
 '날씨': 10,
 '주': 11,
 '지금': 12,
 '해줘': 13,
 '시간': 14,
 '뭐': 15,
 '노래': 16,
 '추천': 17,
 '상태': 18,
 '들려줘': 19,
 '몇': 20,
 '말': 21,
 '공기': 22,
 '이슈': 23,
 '요': 24,
 '무슨': 25,
 '좀': 26,
 '이번': 27,
 '맛집': 28,
 '요즘': 29,
 '나': 30,
 '알려줄래': 31,
 '주변': 32,
 '내일': 33,
 '해줘요': 34,
 '알려주라': 35,
 '지': 36,
 '며칠': 37,
 '알려줄래요': 38,
 '의': 39,
 '좋은': 40,
 '뉴스': 41,
 '사건': 42,
 '현재': 43,
 '이제': 44,
 '곧': 45,
 '번역': 46,
 '날짜': 47,
 '에요': 48,
 '에게': 49,
 '되는': 50,
 '인가요': 51,
 '뭔': 52,
 '거': 53,
 '핫': 54,
 '미세먼지': 55,
 '하나': 56,
 '시야': 57,
 '니': 58,
 '가장': 59,
 '화제': 60,
 '어디': 61,
 '중국어': 62,
 '있나요': 63,
 '이야': 64,
 '요일': 65,
 '인지': 66,
 '힘': 67,
 '들려줘요': 68,
 '일': 69,
 '에': 70,
 '마스크': 71,
 '어떻게': 72,
 '우산': 73,
 '해주라': 74,
 '가': 75,
 '해주세요': 76,
 '멋진': 77,
 '결과': 78,
 '바람': 79,
 '프랑스어': 80,
 '다음': 81,
 '있니': 82,
 '전': 83,
 '누군지': 84,
 '알려주세요': 85,
 '라': 86,
 '영어': 87,
 '음악': 88,
 '시': 89,


# 3. data preprocessing

In [8]:
label_to_id = {t:i for i,t in enumerate(df.intent.unique())}
id_to_label = {i:t for i,t in enumerate(df.intent.unique())}

In [9]:
id_to_label

{0: '음악',
 1: '번역',
 2: '뉴스',
 3: '명언',
 4: '달력',
 5: '먼지',
 6: '인물',
 7: '시간',
 8: '이슈',
 9: '날씨',
 10: '맛집'}

In [10]:
MAX_LENGTH = 40
def tokenize_and_filter(sentences, labels):
  inputs, outputs = [], []
  
  for sentence, label in zip(sentences, labels):
    # tokenize sentence
    tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
    # check tokenized sentence max length
    if len(tokenized_sentence) <= MAX_LENGTH:
      inputs.append(tokenized_sentence)
      outputs.append(label_to_id[label])
  
  # pad tokenized sentences
  padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      inputs, maxlen=MAX_LENGTH, padding='post', 
      value = tokenizer.vocabulary_['_PAD_']) # value = 0
  
  return padded_inputs, outputs

In [11]:
inputs, outputs = tokenize_and_filter(df.question, df.intent)

In [12]:
print('encoded input : ', inputs[0], 'label : ', outputs[0], 'original input sentence : ', tokenizer.decode_from_list(inputs[0]))

encoded input :  [185  73  19   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0] label :  0 original input sentence :  ['에픽하이', '우산', '들려줘', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_', '_PAD_']


In [13]:
BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
for x, y in dataset.take(1):
    print(x, y)
    print('-----------------------------------------------')
    print(x.shape, y.shape)

tf.Tensor(
[[  30    6   26  100    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  29   54  115   23    9    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  77  176   34    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [ 149  202   78    9    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0]
 [  27   11   10   97    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0 

# 4. model design

In [15]:
print(len(label_to_id.values()))

11


In [16]:
def get_model():
    return tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.n_vocabs, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_to_id.values()), activation='softmax')
])

In [17]:
model = get_model()

In [18]:
LEARNING_RATE = 0.0001

In [19]:
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [20]:
model.fit(dataset, epochs=10)

Epoch 1/10
225/225 [==============================] - 8s 38ms/step - loss: 2.3807 - sparse_categorical_accuracy: 0.1242
Epoch 2/10
225/225 [==============================] - 6s 27ms/step - loss: 2.2094 - sparse_categorical_accuracy: 0.3105
Epoch 3/10
225/225 [==============================] - 6s 27ms/step - loss: 1.5460 - sparse_categorical_accuracy: 0.6672
Epoch 4/10
225/225 [==============================] - 6s 27ms/step - loss: 1.0225 - sparse_categorical_accuracy: 0.8516
Epoch 5/10
225/225 [==============================] - 6s 28ms/step - loss: 0.6726 - sparse_categorical_accuracy: 0.9346
Epoch 6/10
225/225 [==============================] - 6s 28ms/step - loss: 0.4475 - sparse_categorical_accuracy: 0.9552
Epoch 7/10
225/225 [==============================] - 6s 28ms/step - loss: 0.3115 - sparse_categorical_accuracy: 0.9710
Epoch 8/10
225/225 [==============================] - 6s 27ms/step - loss: 0.2221 - sparse_categorical_accuracy: 0.9797
Epoch 9/10
225/225 [====================

In [21]:
def question_processing(sentences):
    inputs = []
    for sentence in sentences:
        # tokenize sentence
        tokenized_sentence = tokenizer.encode_a_doc_to_list(sentence)
        # check tokenized sentence max length
        if len(tokenized_sentence) <= MAX_LENGTH:
            inputs.append(tokenized_sentence)
        else:
            print('입력이 너무 길어요.')
    # pad tokenized sentences
    padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    inputs, maxlen=MAX_LENGTH, padding='post', 
    value = tokenizer.vocabulary_['_PAD_']) # value = 0
    return padded_inputs

In [22]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '이순신 장군은 어떤 분이니?',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [23]:
model.predict(input_sentence)

array([[3.3913907e-02, 3.8175474e-04, 3.3587371e-03, 3.0939284e-06,
        5.9731095e-04, 1.3948370e-02, 5.5055339e-06, 1.9805663e-07,
        7.0491456e-07, 9.4541907e-01, 2.3714271e-03],
       [1.2016560e-01, 5.9382775e-04, 5.4020653e-03, 5.3703949e-05,
        3.0661416e-03, 3.0298822e-03, 4.8764928e-06, 2.7779643e-06,
        4.4299776e-07, 8.6604422e-01, 1.6365508e-03],
       [2.2527718e-03, 2.4434816e-02, 1.2365759e-01, 3.3111421e-03,
        4.4613713e-03, 4.1415058e-03, 7.4164480e-01, 2.3803756e-02,
        1.7944578e-02, 6.2991661e-04, 5.3717792e-02],
       [6.9524162e-03, 1.1055868e-01, 2.4180527e-01, 1.3723835e-02,
        2.3659691e-01, 2.6755594e-03, 5.1567040e-02, 2.8376016e-01,
        4.7570989e-02, 5.3829397e-04, 4.2508934e-03],
       [1.6948736e-01, 4.4723932e-02, 3.7512951e-02, 2.0705439e-02,
        3.9333210e-04, 7.6337932e-03, 2.0331221e-02, 1.7625947e-03,
        2.2104169e-04, 1.1006074e-02, 6.8622226e-01]], dtype=float32)

In [24]:
prediction = np.argmax(model.predict(input_sentence), axis=1)
print(prediction)

[ 9  9  6  7 10]


In [25]:
for p in prediction:
    print(id_to_label[p])

날씨
날씨
인물
시간
맛집


# 모델 저장하기

In [26]:
import pickle

model.save("saved_intent_model.h5")
with open('id_to_intent.pickle', 'wb') as handle:
    pickle.dump(id_to_label, handle)

# 데이터 추가해보기

In [27]:
names = ['안중근', '이순신', '세종대왕', '김광석', '아이유', '에미넴', '이건희', '고아라', '유재석', '한석희', '최민성']
def question_generator(names):
    question = []
    for name in names:
        s1 = name+'는 어떤 분이야?'
        s2 = name+'은 어떤 사람이니?'
        s3 = name+'이란 사람에 대해 궁금해'
        question = question+[s1, s2, s3]
    return question
question = question_generator(names)

In [28]:
question

['안중근는 어떤 분이야?',
 '안중근은 어떤 사람이니?',
 '안중근이란 사람에 대해 궁금해',
 '이순신는 어떤 분이야?',
 '이순신은 어떤 사람이니?',
 '이순신이란 사람에 대해 궁금해',
 '세종대왕는 어떤 분이야?',
 '세종대왕은 어떤 사람이니?',
 '세종대왕이란 사람에 대해 궁금해',
 '김광석는 어떤 분이야?',
 '김광석은 어떤 사람이니?',
 '김광석이란 사람에 대해 궁금해',
 '아이유는 어떤 분이야?',
 '아이유은 어떤 사람이니?',
 '아이유이란 사람에 대해 궁금해',
 '에미넴는 어떤 분이야?',
 '에미넴은 어떤 사람이니?',
 '에미넴이란 사람에 대해 궁금해',
 '이건희는 어떤 분이야?',
 '이건희은 어떤 사람이니?',
 '이건희이란 사람에 대해 궁금해',
 '고아라는 어떤 분이야?',
 '고아라은 어떤 사람이니?',
 '고아라이란 사람에 대해 궁금해',
 '유재석는 어떤 분이야?',
 '유재석은 어떤 사람이니?',
 '유재석이란 사람에 대해 궁금해',
 '한석희는 어떤 분이야?',
 '한석희은 어떤 사람이니?',
 '한석희이란 사람에 대해 궁금해',
 '최민성는 어떤 분이야?',
 '최민성은 어떤 사람이니?',
 '최민성이란 사람에 대해 궁금해']

In [29]:
new_data = {'question' : question, 'intent' : ['인물']*len(question)}
add_df = pd.DataFrame(new_data, columns=('question', 'intent'))

In [30]:
add_df.head(5)

,question,intent
0,안중근는 어떤 분이야?,인물
1,안중근은 어떤 사람이니?,인물
2,안중근이란 사람에 대해 궁금해,인물
3,이순신는 어떤 분이야?,인물
4,이순신은 어떤 사람이니?,인물


In [31]:
print(len(df), len(add_df))

3591 33


In [32]:
new_df = pd.concat([df, add_df])
print(len(new_df))

3624


In [33]:
tokenizer.fit(new_df['question'].values)

scanning was done                                        
1413 terms are recognized


In [34]:
new_inputs, new_outputs = tokenize_and_filter(new_df.question, new_df.intent)

BATCH_SIZE = 16
BUFFER_SIZE = 7836

# decoder inputs use the previous target as input
# remove START_TOKEN from targets
dataset = tf.data.Dataset.from_tensor_slices((new_inputs, new_outputs))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [35]:
new_model = get_model()
LEARNING_RATE = 0.0001
new_model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
new_model.fit(dataset, epochs=10)

Epoch 1/10
227/227 [==============================] - 9s 39ms/step - loss: 2.3841 - sparse_categorical_accuracy: 0.2208
Epoch 2/10
227/227 [==============================] - 6s 28ms/step - loss: 2.2262 - sparse_categorical_accuracy: 0.3629
Epoch 3/10
227/227 [==============================] - 6s 28ms/step - loss: 1.5886 - sparse_categorical_accuracy: 0.6388
Epoch 4/10
227/227 [==============================] - 6s 27ms/step - loss: 1.0611 - sparse_categorical_accuracy: 0.8295
Epoch 5/10
227/227 [==============================] - 6s 27ms/step - loss: 0.7302 - sparse_categorical_accuracy: 0.9180
Epoch 6/10
227/227 [==============================] - 6s 27ms/step - loss: 0.4943 - sparse_categorical_accuracy: 0.9473
Epoch 7/10
227/227 [==============================] - 6s 27ms/step - loss: 0.3430 - sparse_categorical_accuracy: 0.9614
Epoch 8/10
227/227 [==============================] - 6s 27ms/step - loss: 0.2467 - sparse_categorical_accuracy: 0.9708
Epoch 9/10
227/227 [====================

In [36]:
input_sentence = question_processing(['서울 날씨 어때?', 
                                      '나는 전주 날씨 궁금함',
                                      '안중근 의사는 누구야?',
                                      '박소희는 어떤 사람인지 궁금해.',
                                      '명동 맛있는 음식점 있니?'
                                     ])

In [37]:
new_model.predict(input_sentence)

array([[4.9589213e-02, 1.2809281e-02, 2.7919335e-03, 3.8396262e-05,
        1.6206429e-04, 2.8144280e-04, 7.2826454e-03, 3.6325629e-03,
        5.7560985e-04, 9.2276138e-01, 7.5480042e-05],
       [7.8573801e-02, 1.5017393e-02, 1.3839473e-03, 9.5085154e-05,
        4.4474233e-04, 1.7480650e-04, 2.0139690e-02, 1.9588403e-02,
        4.4169917e-04, 8.6411631e-01, 2.4144043e-05],
       [3.6772344e-02, 4.7681931e-02, 1.4293131e-03, 1.2719001e-02,
        1.1210305e-02, 1.5252599e-05, 8.8609505e-01, 1.2578344e-03,
        4.3560646e-04, 2.2382506e-03, 1.4516020e-04],
       [4.6258370e-04, 1.0291640e-02, 3.7665705e-03, 9.9312933e-04,
        5.1564139e-01, 1.6445718e-03, 4.5726588e-01, 7.9946313e-03,
        1.6727225e-05, 1.8878463e-03, 3.5130801e-05],
       [2.8394945e-02, 1.8452832e-02, 1.2928452e-01, 1.9538615e-02,
        1.8960953e-05, 6.8824222e-05, 7.3304982e-03, 1.7405448e-06,
        2.2738019e-01, 3.3468247e-04, 5.6919414e-01]], dtype=float32)

In [38]:
prediction = np.argmax(new_model.predict(input_sentence), axis=1)

In [39]:
for p in prediction:
    print(id_to_label[p])

날씨
날씨
인물
달력
맛집
